In [1]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from datetime import datetime
import os

from lstm.model import create_model
from lstm.loader import load_data
from lstm.utils import SlicingDirection
from lstm.process_results import imgs_to_voi, voi_to_object_pcd, calculate_performance_metrics_from_pcds
import lstm.config as config

# Load data
rc_test_dataset, gt_test_dataset = load_data(config.RC_TEST_DATA_PATH, config.GT_TEST_DATA_PATH)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2023-06-13 11:38:42.676742: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 11:38:43.330782: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


shape after loading: (20, 100, 64, 64) (20, 100, 64, 64)
shape after expand dimesnions: (20, 100, 64, 64, 1) (20, 100, 64, 64, 1)


Load model

In [2]:

# Load model from checkpoints
model = create_model() #(None, *x_train.shape[2:])

# Restore the weights
model.load_weights(config.CHECKPOINT_DIR)
#model.summary()

2023-06-13 11:38:46.470736: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-13 11:38:46.519371: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-13 11:38:46.519540: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Frame Prediction Visualizations

In [ ]:
# Select a random example from the validation dataset.
choice_index = np.random.choice(range(len(rc_test_dataset)), size=1)[0]

# Select one example from the dataset
choice_index = 4

rc_original_frames = rc_test_dataset[choice_index]
gt_original_frames = gt_test_dataset[choice_index]

#choice_index = 70
print('Choice index', choice_index)
predictions = []

level_names = []

for i in range(5, 100):
    # Extract the model's prediction and post-process it.
    # new_prediction = predict_fn(tf.constant(np.expand_dims(rc_original_frames, axis=0)))
    level_names.append(f'level_{i+1}.png')

    new_prediction = model.predict(np.expand_dims(rc_original_frames[i:i+5, ...], axis=0))
    new_prediction = np.squeeze(new_prediction, axis=0)
    predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

    # Predicted frames
    predictions.append(predicted_frame)

predictions = np.array(predictions)


now = datetime.now()
now_strf = now.strftime("%m-%d-%yT%H-%M")

prediction_dir = os.path.join(config.DATA_DIR, now_strf, 'prediction')
gt_dir = os.path.join(config.DATA_DIR, now_strf, 'gt')
binary_prediction_dir = os.path.join(config.DATA_DIR, now_strf, 'binary_prediction')

os.makedirs(prediction_dir, exist_ok=True)
os.makedirs(gt_dir, exist_ok=True)
os.makedirs(binary_prediction_dir, exist_ok=True)
   
for i, prediction in enumerate(predictions):
    plt.imsave(os.path.join(prediction_dir, level_names[i]), np.squeeze(prediction), cmap='gray')
    plt.imsave(os.path.join(gt_dir, level_names[i]), np.squeeze(gt_original_frames[i+5]), cmap='gray')

    binary_im = np.zeros(shape=np.squeeze(prediction).shape[:2], dtype=int)
    binary_im[(np.squeeze(prediction) > 0.3)] = 1

    plt.imsave(os.path.join(binary_prediction_dir, level_names[i]), binary_im, cmap='gray')

Process results

In [4]:
# images to voi
dt_voi = imgs_to_voi(binary_prediction_dir, config.VOI_SIZE, SlicingDirection.z)
gt_voi = imgs_to_voi(gt_dir, config.VOI_SIZE, SlicingDirection.z)

# voi to open3d pcd
dt_pcd = voi_to_object_pcd(dt_voi, [0, 0, 1]) 
gt_pcd = voi_to_object_pcd(gt_voi, [0, 1, 0]) 

o3d.visualization.draw_geometries([dt_pcd, gt_pcd])

# Calculate performance metrics
calculate_performance_metrics_from_pcds(dt_pcd, gt_pcd)

{'TP': 1853,
 'FP': 20978,
 'FN': 1072,
 'TN': 2026498,
 'accuracy': 0.9892460060251629,
 'recall': 0.6335042735042735,
 'precision': 0.08116157855547282,
 'specificity': 0.9897542144572147}